In [2]:
from bs4 import BeautifulSoup

In [4]:
import requests

In [35]:
content = requests.get('https://www.biat.tn/biat/Fr/cours-de-change_66_127').text 

In [23]:
print(content)

<Response [200]>


In [36]:
soup= BeautifulSoup(content,'lxml')

In [37]:
table_content = soup.find('div', class_='table_conv')
print (table_content)

<div class="table_conv">
<table cellpadding="0" cellspacing="0" width="100%">
<thead>
<tr>
<th class="first">Désignation</th>
<th>Code</th>
<th>Unité</th>
<th>Achat</th>
<th class="last">Vente</th>
</tr>
</thead>
<tbody>
<tr>
<td>Ryal Saoudien</td>
<td>SAR</td>
<td>10</td>
<td>8.42</td>
<td>8.763</td>
</tr>
<tr>
<td>Dollar Canadien</td>
<td>CAD</td>
<td>1</td>
<td>2.389</td>
<td>2.488</td>
</tr>
<tr>
<td>Couronne Danoise</td>
<td>DKK</td>
<td>100</td>
<td>42.143</td>
<td>43.873</td>
</tr>
<tr>
<td>Dirham Emirats Arabe Unis</td>
<td>AED</td>
<td>10</td>
<td>8.534</td>
<td>8.885</td>
</tr>
<tr>
<td>Dollar des Etats-Unis</td>
<td>USD</td>
<td>1</td>
<td>3.165</td>
<td>3.269</td>
</tr>
<tr>
<td>Livre Sterling(2)</td>
<td>GBP</td>
<td>1</td>
<td>3.606</td>
<td>3.755</td>
</tr>
<tr>
<td>Yen Japonais</td>
<td>JPY</td>
<td>1000</td>
<td>21.81</td>
<td>22.706</td>
</tr>
<tr>
<td>Dinar Koweitien</td>
<td>KWD</td>
<td>1</td>
<td>10.15</td>
<td>10.567</td>
</tr>
<tr>
<td>Couronne Norvegienne</td>


In [63]:
print ('les attributs sont:\n')
for content in table_content: 
    change = table_content.find_all('th')
for h in change:
    headers = h.text
    print (headers)

les attributs sont:

Désignation
Code
Unité
Achat
Vente


In [58]:
change = table_content.find_all('tr')
for a in change: 
        currency = a.find_all('td')
        for currency_text in currency:
            print (currency_text.text)

Ryal Saoudien
SAR
10
8.42
8.763
Dollar Canadien
CAD
1
2.389
2.488
Couronne Danoise
DKK
100
42.143
43.873
Dirham Emirats Arabe Unis
AED
10
8.534
8.885
Dollar des Etats-Unis
USD
1
3.165
3.269
Livre Sterling(2)
GBP
1
3.606
3.755
Yen Japonais
JPY
1000
21.81
22.706
Dinar Koweitien
KWD
1
10.15
10.567
Couronne Norvegienne
NOK
100
31.336
32.622
Ryal Quatari
QAR
10
8.546
8.898
Couronne Suedoise
SEK
10
2.929
3.05
Franc Suisse
CHF
10
32.096
33.413
Euro
EUR
1
3.164
3.268
Dinar Bahraini
BHD
1
8.313
8.655
Yuan CHINOIS
CNY
1
.45
.47
